### Берет данные с гит хаба и возвращает rangingValues.csv и githubUsersFull.csv
* __githubUsersFull.csv__ таблица вида(пользователь, языки программирования)
* __rangingValues.csv__ таблица вида(Фолловеры, Просмотры, Ссылка на гитхаб)

In [6]:
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
from collections import defaultdict

In [7]:
union_skills = ['Assembly', 'C#', 'C++', 'CMake', 'CSS', 'Clojure','CoffeeScript',
                'ColdFusion', 'Delphi', 'GLSL', 'Go', 'Groovy', 'HTML',
                'Java','JavaScript', 'Kotlin','Lua','Objective-C','PHP', 'Perl','PowerShell','Puppet',
                'Python', 'Ruby', 'Smarty', 'Swift', 'TypeScript', 'XML',
                'XSLT']

In [26]:
def getDataFromGit(raw_data):
    """
    Фунция преобразующая данные из 1-го словаря, возвращаемого функцией getData. 
    В разреженную матрицу, где столбцы - языки программирования, строки - пользователи
    пересечение - 0, если пользователь не программирует на этом языке или оценка его способностей в этом языке
    
    Возвращает DataFrame.
    """
    ans = pd.DataFrame(columns = union_skills)
    skills = dict(enumerate(union_skills))
    for user in tqdm(raw_data.values()):
        row = np.zeros(29)
        for i in range(len(union_skills)):
            if skills[i] in user[0].keys():
                row[i] = float(user[0][skills[i]])
            else: 
                row[i] = 0.
        ans = ans.append(pd.DataFrame(np.array(row).reshape(1,-1), columns = union_skills))
    return ans

In [27]:
def getData():
    """
    Функция читающая данные из new_data.json и преобразует это в 2 словаря
    
    Возвращает данные в формате 
    {user_id :[ language: cost]} ,{user_id: [Fllowers, Wacthers, Link]}
    """
    
    with open("new_data.json",'r') as f:
        data = f.read()
    data = data.translate(None, b"\n")
    
    ranging ={}
    data_dict = defaultdict(list)
    
    data_split = data.split('],[')
    for j in range(len(data_split)):
        split = data_split[j].split(' ')
        for i in xrange(len(split)):
            if not split[i]: continue
            if split[i] == '},':
                ranging[j] = [split[i+1][:-1],split[i+2][:-1],split[i+3][1:-1]]
            if split[i][-1] == ":":
                data_dict[j].append({split[i][1:-2]:split[i+1].translate(None,b', ')}) 
    
    return data_dict, ranging

In [28]:
dict_data, ranging = getData()
frame_data = getDataFromGit(dict_data)
rang = pd.DataFrame(np.array(ranging.values()),columns = ["Followers","Watchers","Link"])

100%|██████████| 24293/24293 [00:35<00:00, 675.99it/s]


In [29]:
frame_data.to_csv("newGitHubUsers.csv",index=False)
rang.to_csv("rangingGitLink.csv",index=False)

# Что же получилось

In [30]:
rang.head()

,Followers,Watchers,Link
0,0,0,https://github.com/mcollina
1,82,337,https://github.com/tosch
2,54,227,https://github.com/kennethkalmer
3,30,81,https://github.com/anb
4,32,41,https://github.com/mschiller


In [31]:
frame_data.head()

,Assembly,C#,C++,CMake,CSS,Clojure,CoffeeScript,ColdFusion,Delphi,GLSL,...,Perl,PowerShell,Puppet,Python,Ruby,Smarty,Swift,TypeScript,XML,XSLT
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.5234,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
